In [62]:
#loading packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from scipy.stats import t, shapiro, kstest
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

In [63]:
protein_data = pd.read_csv('data/simko2_data/passport_prots.csv', index_col=0) 
protein_data.index = protein_data.index.astype(str)
protein_data

,22RV1,23132-87,42-MG-BA,451Lu,5637,639-V,647-V,697,769-P,786-0,...,WSU-NHL,YAPC,YH-13,YKG-1,YMB-1-E,YT,ZR-75-30,huH-1,no-10,no-11
protein,,,,,,,,,,,,,,,,,,,,,
A1CF,2.09,2.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.12,NaN,NaN
A2M,2.38,NaN,2.58,NaN,2.79,2.07,3.96,NaN,2.78,2.95,...,0.61,3.00,2.21,2.41,2.45,3.12,2.62,1.06,3.75,3.38
A2ML1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAS,3.05,3.18,2.69,4.21,4.07,3.21,3.76,3.15,3.44,2.55,...,3.46,2.87,3.72,3.36,NaN,2.97,3.22,3.75,3.74,3.02
AACS,2.37,1.42,2.50,3.49,NaN,1.96,NaN,NaN,2.08,NaN,...,NaN,NaN,1.70,NaN,3.44,3.70,2.00,-1.34,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWILCH,2.38,2.08,1.61,NaN,NaN,2.88,NaN,2.68,2.77,1.70,...,1.84,1.02,1.63,3.00,NaN,2.08,NaN,NaN,NaN,3.48
ZWINT,1.69,1.85,1.74,NaN,NaN,2.11,NaN,2.27,2.20,1.82,...,2.00,NaN,1.97,2.98,NaN,1.96,1.18,NaN,1.63,2.71
ZYX,3.60,4.74,5.95,4.34,1.40,6.03,2.50,3.24,7.01,6.61,...,4.78,2.44,7.71,5.62,3.90,6.10,4.56,3.59,6.23,6.07


In [64]:
#removing proteins with over 75% nan
row_nan_count = protein_data.isna().sum(axis=1)
row_nan_percent = (row_nan_count/protein_data.shape[1])*100

#selecting rows with under 80% nan specifically
filt_prot_data = protein_data.dropna()

#trasnpose
filt_prot_data = filt_prot_data.T
filt_prot_data

protein,ABCB6,ACAT1,ACIN1,ACLY,ACO2,ACOT7,ACTBL2,ACTL6A,ACTN4,ADAR,...,YWHAQ,YWHAZ,ZC3H11A,ZC3H14,ZC3H4,ZCCHC8,ZFPL1,ZMPSTE24,ZNF207,ZNF638
22RV1,9.44,7.67,6.16,5.73,7.09,4.70,5.33,4.38,6.99,6.50,...,6.67,9.29,5.36,4.23,3.28,4.46,5.52,4.39,5.57,4.66
23132-87,9.63,7.93,6.06,5.31,6.96,6.03,6.33,3.82,7.33,6.50,...,6.48,10.66,4.16,3.93,3.31,4.57,5.26,5.21,5.85,4.31
42-MG-BA,10.58,6.45,5.70,5.91,6.00,5.45,7.41,4.04,7.70,5.07,...,6.82,9.57,3.86,4.29,3.12,2.77,5.11,6.27,5.41,3.55
451Lu,8.81,8.04,6.37,4.52,8.14,6.24,6.14,4.93,6.67,6.31,...,6.07,9.06,5.18,4.46,3.71,3.51,5.47,3.78,5.17,4.41
5637,5.33,6.05,6.02,2.79,6.21,2.78,5.75,4.48,5.49,6.24,...,3.67,6.23,4.09,4.44,2.24,3.40,5.86,4.94,4.16,4.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,10.25,5.73,5.75,5.78,6.85,6.07,6.44,4.38,6.86,6.44,...,8.35,9.23,4.74,3.63,4.02,3.74,3.87,4.94,5.98,4.07
ZR-75-30,7.89,6.33,5.79,4.81,7.25,4.18,6.96,4.72,7.85,6.65,...,6.69,11.37,5.14,4.09,3.59,3.52,6.33,5.66,5.26,4.35
huH-1,8.30,8.64,6.29,4.55,6.12,2.85,6.45,4.09,6.02,6.54,...,5.19,7.75,4.44,4.05,3.26,3.49,5.63,5.23,5.53,3.89
no-10,9.93,8.05,5.98,6.00,5.72,6.18,6.77,5.22,9.00,5.93,...,6.52,9.10,4.88,4.04,3.64,3.35,6.52,4.19,5.78,4.18


In [65]:
#scaling data
scaler = StandardScaler()
filt_prot_data = pd.DataFrame(scaler.fit_transform(filt_prot_data), index=filt_prot_data.index, columns=filt_prot_data.columns)
filt_prot_data

protein,ABCB6,ACAT1,ACIN1,ACLY,ACO2,ACOT7,ACTBL2,ACTL6A,ACTN4,ADAR,...,YWHAQ,YWHAZ,ZC3H11A,ZC3H14,ZC3H4,ZCCHC8,ZFPL1,ZMPSTE24,ZNF207,ZNF638
22RV1,0.339086,0.584458,0.554077,0.363266,1.009232,-0.369939,-1.749049,-0.454913,-0.021696,0.677770,...,0.078044,0.095058,1.142044,0.130545,-0.615025,1.326619,0.634928,-0.030758,-0.308983,0.686625
23132-87,0.491141,0.903253,0.279758,-0.059999,0.818395,0.745345,-0.300852,-1.493285,0.279723,0.677770,...,-0.107875,1.552639,-0.699367,-0.564196,-0.556796,1.526404,0.341235,0.874230,0.118297,-0.021095
42-MG-BA,1.251418,-0.911428,-0.707788,0.544666,-0.590859,0.258981,1.263201,-1.085353,0.607737,-2.061355,...,0.224822,0.392958,-1.159720,0.269493,-0.925578,-1.742820,0.171797,2.044092,-0.553144,-1.557860
451Lu,-0.165098,1.038128,1.130146,-0.856140,2.550605,0.921443,-0.576010,0.564916,-0.305384,0.313830,...,-0.509068,-0.149646,0.865833,0.663180,0.219586,-0.398806,0.578449,-0.703980,-0.919384,0.181111
5637,-2.950113,-1.401882,0.170031,-2.599589,-0.282585,-1.979974,-1.140807,-0.269490,-1.351483,0.179747,...,-2.857515,-3.160562,-0.806783,0.616864,-2.633619,-0.598591,1.018988,0.576246,-2.460646,0.342875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,0.987322,-1.794245,-0.570629,0.413655,0.656918,0.778888,-0.141551,-0.454913,-0.136944,0.562842,...,1.721957,0.031222,0.190648,-1.258937,0.821282,0.018929,-1.228891,0.576246,0.316678,-0.506389
ZR-75-30,-0.901366,-1.058564,-0.460901,-0.563886,1.244108,-0.805991,0.611512,0.175526,0.740716,0.965091,...,0.097615,2.308028,0.804452,-0.193668,-0.013329,-0.380643,1.549894,1.370870,-0.782044,0.059787
huH-1,-0.573247,1.773810,0.910691,-0.825907,-0.414703,-1.921275,-0.127069,-0.992641,-0.881625,0.754389,...,-1.370165,-1.543392,-0.269705,-0.286300,-0.653844,-0.435130,0.759183,0.896303,-0.370023,-0.870360
no-10,0.731228,1.050390,0.060304,0.635365,-1.001892,0.871129,0.336354,1.102644,1.760220,-0.414049,...,-0.068734,-0.107089,0.405480,-0.309458,0.083719,-0.689403,1.764515,-0.251486,0.011477,-0.283963


In [66]:
# make into tensor
protein_tensor = torch.tensor(filt_prot_data.values, dtype=torch.float32)
protein_tensor

tensor([[ 0.3391,  0.5845,  0.5541,  ..., -0.0308, -0.3090,  0.6866],
        [ 0.4911,  0.9033,  0.2798,  ...,  0.8742,  0.1183, -0.0211],
        [ 1.2514, -0.9114, -0.7078,  ...,  2.0441, -0.5531, -1.5579],
        ...,
        [-0.5732,  1.7738,  0.9107,  ...,  0.8963, -0.3700, -0.8704],
        [ 0.7312,  1.0504,  0.0603,  ..., -0.2515,  0.0115, -0.2840],
        [ 0.7072,  0.0695, -0.6255,  ...,  1.4923,  0.1183, -0.5671]])

In [67]:
#data loader
dataset = TensorDataset(protein_tensor)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [68]:
#defining variational autoencoder
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim  # Number of proteins (9000)
        self.latent_dim = latent_dim  # Compressed representation size

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        
        # Latent space (mean and variance)
        self.fc_mu = nn.Linear(256, latent_dim)  # Mean
        self.fc_logvar = nn.Linear(256, latent_dim)  # Log variance

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim)  # Output same shape as input
        )

    def encode(self, x):
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)  # Convert log variance to standard deviation
        eps = torch.randn_like(std)  # Random noise
        return mu + eps * std  # Sample from the latent space

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

In [69]:
# Loss function (Reconstruction loss + KL divergence)
def vae_loss(x, x_recon, mu, logvar):
    recon_loss = nn.MSELoss()(x_recon, x)  # Reconstruction loss (no scaling needed)
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL divergence
    return recon_loss + kl_div

In [70]:
 #Training loop
def train_vae(model, train_loader, epochs=100, lr=0.0001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for data in train_loader:  # Using your existing dataloader
            data = data[0]
            data = data.to(torch.float32)
            optimizer.zero_grad()
            x_recon, mu, logvar = model(data)
            loss = vae_loss(data, x_recon, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

In [71]:
vae = VAE(input_dim=822, latent_dim=128)

In [72]:
# Train the VAE using your existing dataloader
train_vae(vae, train_loader, epochs=50, lr=0.001)

Epoch 1, Loss: 4.844954133033752
Epoch 2, Loss: 1.4370404918988546
Epoch 3, Loss: 1.1510448972384135
Epoch 4, Loss: 1.0482525924841564
Epoch 5, Loss: 1.0364738762378694
Epoch 6, Loss: 1.0205265959103902
Epoch 7, Loss: 1.0089092155297597
Epoch 8, Loss: 1.0065830628077188
Epoch 9, Loss: 1.007529624303182
Epoch 10, Loss: 1.0007567524909973
Epoch 11, Loss: 1.0032135526339212
Epoch 12, Loss: 0.9994072198867798
Epoch 13, Loss: 1.0010168174902598
Epoch 14, Loss: 1.0016022781531015
Epoch 15, Loss: 0.9987850685914358
Epoch 16, Loss: 1.0022162934144339
Epoch 17, Loss: 1.0042138755321504
Epoch 18, Loss: 1.0011469066143035
Epoch 19, Loss: 1.0012060085932413
Epoch 20, Loss: 1.0007667064666748
Epoch 21, Loss: 1.002431219816208
Epoch 22, Loss: 1.003468664487203
Epoch 23, Loss: 1.0003293037414551
Epoch 24, Loss: 1.0030636847019196
Epoch 25, Loss: 1.0023257593313852
Epoch 26, Loss: 1.00261150598526
Epoch 27, Loss: 1.0006115158398947
Epoch 28, Loss: 1.000399124622345
Epoch 29, Loss: 1.0014509002367655
E